In [18]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key='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',
    model="llama-3.3-70b-versatile",
    temperature=0
    
)

In [19]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-47637?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Software Engineer, Data & Analytics - Converse Technology (Open to ATL, Beaverton, BOS)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software Engineer, 

In [33]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [34]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer, Data & Analytics - Converse Technology',
 'experience': 'Minimum 6 years relevant work experience',
 'skills': ['Strong foundation in data analytics',
  'Experience working with large data sets and defining complex data models',
  'Proven experience leading engineering teams',
  'Experience working with analytics tools such as Tableau, Alteryx, Sigma, etc.',
  'Writing SQL, R, and/or Python code',
  'Familiarity with Agile principles and methodologies (Scrum, Kanban)'],
 'description': 'Lead the creation, improvement and adoption of Converse’s data products, setting the product vision and managing the data engineering roadmap to support the growing analytics discipline at Converse.'}

In [22]:
type(json_res)

dict

In [23]:
import pandas as pd

df = pd.read_csv(r"C:\Users\vatsa\Desktop\Projects\Cold Email generator\app\resources\portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [24]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [26]:
job = json_res

In [27]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}]]

In [35]:
job

{'role': 'Lead Software Engineer, Data & Analytics - Converse Technology',
 'experience': 'Minimum 6 years relevant work experience',
 'skills': ['Strong foundation in data analytics',
  'Experience working with large data sets and defining complex data models',
  'Proven experience leading engineering teams',
  'Experience working with analytics tools such as Tableau, Alteryx, Sigma, etc.',
  'Writing SQL, R, and/or Python code',
  'Familiarity with Agile principles and methodologies (Scrum, Kanban)'],
 'description': 'Lead the creation, improvement and adoption of Converse’s data products, setting the product vision and managing the data engineering roadmap to support the growing analytics discipline at Converse.'}

In [29]:
job['skills']

['Strong foundation in data analytics',
 'Experience working with large data sets and defining complex data models',
 'Proven experience leading engineering teams',
 'Experience working with analytics tools such as Tableau, Alteryx, Sigma, etc.',
 'Writing SQL, R, and/or Python code',
 'Familiarity with Agile principles and methodologies (Scrum, Kanban)']

In [36]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Rahul, a business development executive at PixelMind. PixelMind is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of PixelMind 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase PixelMind's portfolio: {link_list}
        Remember you are Rahul, BDE at PixelMind. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Analytics Solutions for Converse Technology

Dear Hiring Manager,

I came across the job description for a Lead Software Engineer, Data & Analytics at Converse Technology, and I'm excited to introduce PixelMind, a leading AI & Software Consulting company that can help you achieve your data analytics goals. With our expertise in developing tailored solutions, we can empower your team to drive business growth through data-driven decision-making.

At PixelMind, we have a strong foundation in data analytics, and our team has experience working with large data sets, defining complex data models, and leading engineering teams. We're well-versed in analytics tools such as Tableau, Alteryx, and Sigma, and our developers are proficient in writing SQL, R, and Python code. We follow Agile principles and methodologies, including Scrum and Kanban, to ensure seamless project execution.

Our portfolio showcases our capabilities in developing innovative solutions using various tec